In [1]:
# Importing all needed libraries
# Using openstreetmap geocoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re
import requests
import json
from geopy.geocoders import Nominatim # for geocoding
from geopy.distance import geodesic # for distance calculation
import geocoder # to get user ip for its location
import plotly.graph_objects as go # for plotly plotting
import folium # for folium plotting
import branca.colormap as cm # for colormaps in folium
%matplotlib inline

In [2]:
# Question 2 libs
from tqdm import tqdm
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import *
import string

# Question 2 PREREQUISITES TO TEST

In [3]:
# Import data - TEMPORARY
df_courses = pd.read_csv('your_file.csv') # Scraped df

In [4]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load SnowballStemmer
snowstem = SnowballStemmer('english')

# Load stopwords to filter
lst_stopwords = set(stopwords.words('english'))

# Define the function to preprocess text
def preprocess_text(text):
    
    # FIX - HANDLING NAN VALUES
    try:
        text_lower = text.lower()
    except:
        return text
    
    # ORIGINAL CODE
    # Convert to lowercase this is because for some reason it wasn't elimination from stopword the word 'the' when starting with an uppercase
    #text_lower = text.lower()

    # Tokenize, stem words, remove punctuation and remove stopwords
    stemmed_words = [snowstem.stem(word) for word in nltk.word_tokenize(text_lower) if not word in lst_stopwords and word.isalnum()]

    return stemmed_words

# Create column named 'descr_clean' and then apply preprocess_text to the 'Description' column in df_courses
df_courses['descr_clean'] = df_courses['Description'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /home/gab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/gab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Where GBP is the base currency you want to use
# Store API Key in another file
url = 'https://v6.exchangerate-api.com/v6/ca6656c9b54065b55c44b78b/latest/GBP'

# Making our request
response = requests.get(url)
data_exchange = response.json()

with open('data_exchange.json', 'w') as json_file:
    json.dump(data_exchange, json_file)

# Read the JSON data from the file
with open('data_exchange.json', 'r') as json_file:
    loaded_data = json.load(json_file)

# Convert the 'result' column to a Pandas DataFrame
df = pd.DataFrame(loaded_data)

# Extract only the 'conversion_rates' column
conversion_rates = df['conversion_rates']

In [7]:
# Defined a function to select the maximum fee from a list of fees
def max_value_fee(list_fees):
    filtered_fee_list = [int(fee) for fee in list_fees]
    if filtered_fee_list:
        max_fee = max(filtered_fee_list)
        max_fee = float(max_fee)
    else:
        # If the list is empty, set max_fee to None
        max_fee = None
    return max_fee

# Define a function to remove dots and commas from numbers in a string
def remove_dots_from_numbers(input_string):
    # Use regular expression to find numbers with dots or commas and remove the dots or commas
    result_string = re.sub(r'(\d)[.,](\d{3})', r'\1\2', input_string)
    return result_string

# Iterate over each row in df_courses
for index, row in df_courses.iterrows():
    # Extract the 'Fees' column as a string
    fees_string = str(row['Fees'])

    # Remove dots from numbers and specific year strings (These years tend to create problems)
    fees_string = remove_dots_from_numbers(fees_string)
    fees_string = fees_string.replace("2022", '').replace("2023", '').replace("2024", '')

    # Remove various punctuation, spaces, and special characters
    fees_string = fees_string.replace(".00 ", '').replace(".00€ ", '').replace(",00 ", '').replace(",00€ ", '').replace(".0 ", '').replace(",0 ", '').replace('.', '').replace(',', '').replace("'", '').replace(" ", '')

    # Use regular expression to find currency symbols or codes ISO 4217 for all currencies (Those are the once that are used in our conversion_rates variable)
    matches_cur = re.findall(r'HKD(?:s)?|HK(?:s)?|\p{Sc}|euro(?:s)?|dollar(?:s)?|pound(?:s)?|EUR(?:s)?|USD(?:s)?|CHF(?:s)?|SEK(?:s)?|ISK(?:s)?|RMB(?:s)?|QR(?:s)?|GBP(?:s)?|QAR(?:s)?|JPY(?:s)?', fees_string, flags=re.IGNORECASE)

    # Use regular expression to find fees (excluding specific years) in the cleaned string
    matches_fee = re.findall(r'(?!2021|2022|2023|2024)\d{4,}', fees_string)

    # Call the max_value_fee function to get the maximum fee
    fees_float = max_value_fee(matches_fee)

    # Check if currency matches and assign the corresponding ISO 4217 code for all currencies (Those are the once that are used in our conversion_rates variable)
    if matches_cur:
        currency = None
        if matches_cur[0] == 'euro' or matches_cur[0] == 'euros' or matches_cur[0] == '€' or matches_cur[0] == 'EURs' or matches_cur[0] == 'Eur' or matches_cur[0] == 'Euros' or matches_cur[0] == 'EUROS' or matches_cur[0] == 'eurs'or matches_cur[0] == 'Euro':
            currency = 'EUR'
        elif matches_cur[0] == 'dollar' or matches_cur[0] == 'dollars' or matches_cur[0] == '$':
            currency = 'USD'
        elif matches_cur[0] == 'pound' or matches_cur[0] == 'pounds' or matches_cur[0] == '£':
            currency = 'GBP'
        elif matches_cur[0] == 'RMB':
            currency = 'CNY'
        elif matches_cur[0] == 'QR':
            currency = 'QAR'
        elif matches_cur[0] == 'HK':
            currency = 'HKD'
        else:
            currency = matches_cur[0]
    else:
        currency = None

    # Initialize fees_pound as None
    fees_pound = None

    # Check if fees_float is not None and less than 100000 before rounding and conversion (fees (£) above 100000 are due to super specific errors, less than 10 which are more reasonable to deal in this way)
    if fees_float is not None and fees_float < 100000:
        fees_pound = round(fees_float / conversion_rates.get(currency, 1.0), 2)

    # Assign the calculated fees in pounds to the 'fees (£)' column
    df_courses.at[index, 'fees (£)'] = fees_pound

# Filter the df_cources to include only rows where 'fees (£)' is not null - EDITED FOR TESTING
dforig = df_courses[(df_courses['fees (£)'].notnull())]
display(dforig.shape)
display(dforig)

# ORIGINAL CODE
#filtered_df = df_courses[(df_courses['fees (£)'].notnull())]
#display(filtered_df.shape)

(1354, 13)

,Course Name,University Name,Faculty Name,Description,Fees,Modality,Duration,City,Country,Link,administration,descr_clean,fees (£)
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Businesses and governments rely on sound finan...,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[busi, govern, reli, sound, financi, knowledg,...",34750.00
5,Advanced Chemical Engineering - MSc,University of Leeds,School of Chemical and Process Engineering,The Advanced Chemical Engineering MSc at Leeds...,"UK: £13,750 (Total) International: £31,000 (To...",MSc,1 year full time,Leeds,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[advanc, chemic, engin, msc, leed, build, core...",31000.00
7,Agricultural Sciences - MSc (Agriculture and F...,University of Helsinki,International Masters Degree Programmes,Goal of the pro­grammeWould you like to be inv...,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,https://www.findamasters.com/masters-degrees/c...,On Campus,"[goal, like, involv, find, solut, futur, chall...",13068.48
8,"Agricultural, Environmental and Resource Econo...",University of Helsinki,International Masters Degree Programmes,Goal of the pro­grammeAre you looking forward ...,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,https://www.findamasters.com/masters-degrees/c...,On Campus,"[goal, look, forward, futur, expert, agricultu...",13068.48
9,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Up to 7 million people are estimated to die ev...,"UK: £12,500 (Total) International: £28,750 (To...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[7, million, peopl, estim, die, everi, year, d...",28750.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5983,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Harbour.Space's FinTech Master programme is de...,"€29,900/year","MBA, MSc",1 Year,Barcelona,Spain,https://www.findamasters.com/masters-degrees/c...,On Campus,"[fintech, master, programm, design, prepar, gr...",26049.83
5984,Master's of Front-end Development,Harbour.Space University,Masters Programmes,Front-end Development at Harbour.Space Univers...,"€29,900/year",MSc,1 year,Barcelona,Spain,https://www.findamasters.com/masters-degrees/c...,On Campus,"[develop, univers, provid, uniqu, environ, stu...",26049.83
5992,Materials and Molecular Modelling MSc,University College London,Department of Chemistry,Register your interest in graduate study at UC...,"Full time - £14,100",MSc,1 year full time,London,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[regist, interest, graduat, studi, uclther, gr...",14100.00
5995,Materials Engineering,University of Padua,School of Engineering,The Master's degree Materials Engineering is a...,Our tuition fees will not exceed 2700 euros pe...,MSc,2 years,Padua,Italy,https://www.findamasters.com/masters-degrees/c...,On Campus,"[master, degre, materi, engin, interdisciplina...",2352.33


# Question 4

#### Using maps can help people understand how far one university is from another so they can plan their academic careers more adequately. Here, we challenge you to show a map of the courses found with the score defined in point 3. You should be able to identify at least the city and country for each MSc degree. <br> Once you defined your visualization strategy, include a way to encode fees in your charts. The map should show (with a proper legend) different courses and associated taxation: the user wants a glimpse not only of how far he will need to move but also of how much it will cost him!

The objective is to plot on a map univeristies that host the suggested courses, along with some useful informations about them (like distance and cost).  
The *Folium* python library has been chosen to do so, because it provides a detailed map and the right amount of interactivity: the user will be able to move and zoom across the map, and also visualize the courses names along with the associated fees. 

In [8]:
##### TEMPORARY + PREPROCESSING #####
dfuniv = dforig[['Course Name', 'University Name', 'Description', 'Link']]
print(dfuniv.shape)

k = 40 # Number of courses requested

qdf = dfuniv[0:k].copy() # After query df
display(qdf.head())

(1354, 4)


,Course Name,University Name,Description,Link
1,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound finan...,https://www.findamasters.com/masters-degrees/c...
5,Advanced Chemical Engineering - MSc,University of Leeds,The Advanced Chemical Engineering MSc at Leeds...,https://www.findamasters.com/masters-degrees/c...
7,Agricultural Sciences - MSc (Agriculture and F...,University of Helsinki,Goal of the pro­grammeWould you like to be inv...,https://www.findamasters.com/masters-degrees/c...
8,"Agricultural, Environmental and Resource Econo...",University of Helsinki,Goal of the pro­grammeAre you looking forward ...,https://www.findamasters.com/masters-degrees/c...
9,Air Quality Solutions - MSc,University of Leeds,Up to 7 million people are estimated to die ev...,https://www.findamasters.com/masters-degrees/c...


**Geocoding** universities coordinates is achieved by using *Geopy*, a library which provides simple and fast access to numerous geocoding *APIs*. Precisely, it was decided to use *Nominatim*: it's a free *API* which relies on the *OSM (OpenStreetMap)* service.  
For each query (e.g. university name) it returns a dictionary with related informations, such as the city, address, longitude, latitude, etc.  
Because it is free, there are some cons:
- It can handle only 1 request per second, making it not suitable for bulk geocoding;
- It doesn't handle typos or traductions, resulting in not geocoding the coordinates of some universities. When it is the case, the coordinates can be estimated by referring to the city (or the country) the respective university lies in.

In [9]:
# GEOCODING

# include city and fees for each (suggested) course in the dataframe
qdf['fees'] = pd.concat([qdf, dforig], axis=1, join="inner")['fees (£)']
qdf['city'] = pd.concat([qdf, dforig], axis=1, join="inner")['City']
qdf['country'] = pd.concat([qdf, dforig], axis=1, join="inner")['Country']

# geocode university coordinates, add them to dataframe and check the output
gc = Nominatim(timeout = 1, user_agent="adm_hw3")
coords = qdf['University Name'].apply(gc.geocode, exactly_one = True)
# geocode city coordinates when the university could not be geocoded
coords.fillna(qdf['city'].apply(gc.geocode, exactly_one = True), inplace = True)
# geocode country coordinates when the city could not be geocoded
coords.fillna(qdf['country'].apply(gc.geocode, exactly_one = True), inplace = True)

# create the proper columns in the dataset
qdf['lat'] = np.array([l.latitude for l in coords])
qdf['lon'] = np.array([l.longitude for l in coords])

display(qdf.head()) # CHECK

,Course Name,University Name,Description,Link,fees,city,country,lat,lon
1,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound finan...,https://www.findamasters.com/masters-degrees/c...,34750.00,Leeds,United Kingdom,53.806774,-1.556288
5,Advanced Chemical Engineering - MSc,University of Leeds,The Advanced Chemical Engineering MSc at Leeds...,https://www.findamasters.com/masters-degrees/c...,31000.00,Leeds,United Kingdom,53.806774,-1.556288
7,Agricultural Sciences - MSc (Agriculture and F...,University of Helsinki,Goal of the pro­grammeWould you like to be inv...,https://www.findamasters.com/masters-degrees/c...,13068.48,Helsinki,Finland,60.175648,24.953550
8,"Agricultural, Environmental and Resource Econo...",University of Helsinki,Goal of the pro­grammeAre you looking forward ...,https://www.findamasters.com/masters-degrees/c...,13068.48,Helsinki,Finland,60.175648,24.953550
9,Air Quality Solutions - MSc,University of Leeds,Up to 7 million people are estimated to die ev...,https://www.findamasters.com/masters-degrees/c...,28750.00,Leeds,United Kingdom,53.806774,-1.556288


Some **preprocessing** must be done before plotting the map. As some of the suggested courses could be provided by the same university, it was decided to **aggregate** rows with the same university name (keeping courses names and the respective fees in lists).  
A column named **label** was added, which contains the formatted labels that will be used to show courses informations in the map. To give fees infos at a glimpse, a column named **minfee** was added, which contains the minimum annual fee among the (suggested) courses provided by the same university.  
Lastly, the user location is geocoded to calculate and store in the **dist** column the distance between him and the universities.

In [10]:
# PLOTTING PREPROCESSING -- Folium
# Remember that more than one course per university can be returned
# so make sure that when hovering on a university all infos are printed 

# Aggregate rows with the same university name, creating a list of courses for each university,
# with the associated lists including descriptions, urls and fees
aggregation_functions = {'Course Name': list, 'fees': list, 
                         'Link': list, 'lat': 'first', 'lon': 'first', 'city': 'first',
                         'country': 'first'}
pdf = qdf.groupby(qdf['University Name'], as_index=False).aggregate(aggregation_functions)

# Create a 'label' column, which contains the formatted labels that will be used in the map
pdf['label'] = pdf.apply(lambda row: '<br>'.join([i + ' (' + str(j) + ' £/y)'  for i, j in zip(row['Course Name'], row['fees'])]), axis = 1)

# Create a 'minfee' column, which will be used in the map for color assignment
pdf['minfee'] = pdf['fees'].apply(np.min)

# Geocode user location, handling the case which it can't be done (default location will be Rome)
try:
    user_ip = geocoder.ip('me')
except:
    user_ip = pd.DataFrame(np.array([41.902782, 12.496366]), columns = ['latlng'])
# Create a 'dist' column, which stores the distance (km) from user
pdf['dist'] = pdf.apply(lambda row: round(geodesic([row['lat'],row['lon']],user_ip.latlng).km,3), axis = 1)

display(pdf.head()) # CHECK

,University Name,Course Name,fees,Link,lat,lon,city,country,label,minfee,dist
0,Erasmus School of Economics,"[Economics of Markets and Organisations, Econo...","[14026.83, 14026.83]",[https://www.findamasters.com/masters-degrees/...,51.924442,4.477750,Rotterdam,Netherlands,Economics of Markets and Organisations (14026....,14026.83,1270.405
1,Learna | Diploma MSc,[Care of the Older Person - MSc],[8880.0],[https://www.findamasters.com/masters-degrees/...,51.481655,-3.179193,Cardiff,United Kingdom,Care of the Older Person - MSc (8880.0 £/y),8880.00,1599.269
2,Northumbria University,"[Business and Management MSc (Online), Compute...","[9075.0, 9075.0, 9075.0]",[https://www.findamasters.com/masters-degrees/...,54.977755,-1.607282,Newcastle,United Kingdom,Business and Management MSc (Online) (9075.0 £...,9075.00,1784.036
3,University College London,[Humanitarian Policy and Practice MSc],[11800.0],[https://www.findamasters.com/masters-degrees/...,51.524203,-0.133001,London,United Kingdom,Humanitarian Policy and Practice MSc (11800.0 ...,11800.00,1438.669
4,University for the Creative Arts,"[Fashion Business & Management - MA/MSc, Globa...","[10500.0, 10500.0]",[https://www.findamasters.com/masters-degrees/...,51.216262,-0.806057,Farnham,United Kingdom,Fashion Business & Management - MA/MSc (10500....,10500.00,1450.803


As said previously, the map was plotted using a library called *Folium*. Each university which provides at least one of the suggested courses is displayed, along with:
- Its name
- Course(s) name(s)
- Course(s) fees
- Distance (as the crow flies) from the user location

The column **minfee** was used to color-code the minimun fees required for each university, going from green (least expensive at minimum) to red (most expensive at minimum).

In [11]:
# PLOTTING -- Folium

# Create colormap based on minimum fee - WIP don't know if it has to start from 0 or from the minimum
colormap = cm.LinearColormap(colors=['green','yellow','red'], vmin=np.min(pdf['minfee']),vmax=np.max(pdf['minfee']),
                             caption='Minimum fees among the university MScs (£/y)')
colormap

# Set initial location and zoom of the map
uni_map = folium.Map(location=user_ip.latlng, zoom_start=6)
# Add user location marker
folium.Marker(location = user_ip.latlng,
              tooltip = 'Your location').add_to(uni_map)
# Add the suggested universities markers and the colormap
pdf.apply(lambda row: folium.CircleMarker(location=[row['lat'],row['lon']], 
                                    color = "black", 
                                    radius = 6,
                                    weight = 2,
                                    fill = True,
                                    fill_color = colormap(row['minfee']),
                                    fill_opacity = 0.8,
                                    tooltip = '<b>Institute</b>: '+ row['University Name']+'<br>' +
                                              '<b>Distance (as the crow flies)</b>: ' + str(row['dist']) + ' km'+ '<br>'+
                                              '<b>Course(s)</b>: '+ row['label']+'<br>' +
                                              '<extra></extra>').add_to(uni_map), axis = 1)
uni_map.add_child(colormap)

# Save the map onto an html file and print it inside the notebook
uni_map.save(outfile='Map - Universities hosting the Top-'+ str(k) +' suggested MScs.html')
display(uni_map)

## MAP PROTOTYPE USING PLOTLY - IGNORE IT

In [8]:
# PLOTTING -- Plotly
# Remember that more than one course per university can be returned
# so make sure that when hovering on a university all infos are printed 

# merge rows with the same university name, creating a list of courses for each university,
# with associated lists of descriptions, urls and fees
aggregation_functions = {'Course Name': lambda x: '<br>                '.join(x), 'fees': lambda x: '<br>          '.join(x), 
                         'Link': lambda x: '<br>'.join(x), 'lat': 'first', 'lon': 'first', 'city': 'first',
                         'country': 'first'}
pdf = qdf.groupby(qdf['University Name'], as_index=False).aggregate(aggregation_functions)

fig = go.Figure(data=go.Scattergeo(
        lat = pdf['lat'],
        lon = pdf['lon'],
        hoverinfo = 'text',
        customdata=np.stack((pdf['University Name'], pdf['Course Name'], pdf['fees']), axis=-1),
        hovertemplate= '<b>Institute</b>: %{customdata[0]}<br>' +
                          '<b>Courses</b>: %{customdata[1]}<br>' +
                          '<b>Fees(curr/y)</b>: %{customdata[2]}<br>' +
                          '<extra></extra>',
        mode = 'markers',
        marker = dict(
            size = 8, # size of markers TO SET
            opacity = 0.8, # opacity of markers TO SET
            reversescale = True, # FIGURE OUT WHAT IT IS
            autocolorscale = False, # FIGURE OUT WHAT IT IS
            symbol = 'square', # markers symbol TO SET
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ), # FIGURE OUT WHAT IT IS
            colorscale = 'Reds', # colorscale TO SET maybe red??
            cmin = 0,
            #color = qdf['fees'], #???
            #cmax = qdf['fees'].max(),
            #colorbar_title="Course fee"
        )))

fig.update_layout(
        title = 'Universities hosting the Top-'+ str(k) +' suggested MScs',
        geo_scope='world',
    )

fig.show()

TypeError: sequence item 0: expected str instance, float found